# SMOLAGENTS

## Overview
This function uses the [SmolAgents](https://huggingface.co/docs/smolagents/main/en/index) library to run a simple code agent that can generate personalized greetings or respond to instructions using an LLM model. It is designed for use in Excel, allowing users to interact with AI agents directly from their spreadsheets.

## Usage
To use the `SMOLAGENTS` function in Excel, enter it as a formula in a cell, specifying your prompt and any optional arguments as needed:

```excel
=SMOLAGENTS(prompt, [model_id], [api_key])
```
Replace each parameter with your desired value. The function returns a text response generated by the agent.

## Parameters
| Parameter | Type   | Required | Description                                                                 | Default                                 |
|-----------|--------|----------|-----------------------------------------------------------------------------|-----------------------------------------|
| prompt    | string | Yes      | The instruction or request for the agent (e.g., "Get a greeting for Mike"). |                                         |
| model_id  | string | No       | The model identifier or endpoint URL to use for inference. See the [SmolAgents InferenceClientModel documentation](https://huggingface.co/docs/smolagents/main/en/reference/models#smolagents.InferenceClientModel) for valid values and usage. | https://api.mistral.ai/v1/chat/completions |
| api_key   | string | No       | The API key for the LLM provider. [Get a free API key from Mistral AI](https://console.mistral.ai/). | Microsoft login token used for demo if not provided. |

## Return Value
| Return Value | Type   | Description                                  |
|--------------|--------|----------------------------------------------|
| Response     | string | The text response generated by the agent, or an error message if something fails. |

## Tools
The agent is configured with a single embedded tool: the `hello` function. This function takes a name as input and returns a personalized greeting. All agent responses are generated by invoking this `hello` tool, so the agent's capabilities are limited to greeting tasks.

## Demo
If either `api_key` or `model_id` is not provided, both will default to Boardflare demo values (`model_id`: https://llm.boardflare.com, `api_key`: your Microsoft login token if available). This only works for users logged in with a Microsoft account and provides limited free demo usage. You may obtain a free api_key for [Mistral AI](https://console.mistral.ai/) with your Microsoft account which offers more generous free usage and supports CORS.

## Limitations
- If you have another function in your workbook that has used `Requests` before you run this function, you may encounter the following error: `ValueError: Requested 'requests>=2.32.3', but requests==2.31.0 is already installed`. Close and reopen Excel, and only use this function in that workbook.  We are working on a fix for this.
- Only string prompts are accepted; invalid or empty prompts return an error message.
- The function is limited to simple greeting tasks in its current form as it only uses the `hello` tool.
- The function requires an internet connection to access the AI model.
- Model availability and output may vary depending on the provider or API changes.

## Examples

### Mistral AI
Request a greeting for a client named Mike using Mistral AI:
```excel
=SMOLAGENTS("Get a greeting for Mike", "https://api.mistral.ai/v1/chat/completions", "your_mistral_api_key")
```

### Demo Mode
If you do not provide an API key, the function will attempt to use the Boardflare demo endpoint (rate-limited):
```excel
=SMOLAGENTS("Get a greeting for Bob")
```

In [ ]:
import micropip
await micropip.install('smolagents')
from smolagents import CodeAgent, InferenceClientModel, tool

def smolagents(prompt, model_id = "https://api.mistral.ai/v1/chat/completions", api_key = None):
    """
    Uses a code agent from smolagents to run a simple tool (hello) using an LLM model.

    Args:
        prompt (str): The instruction or request for the agent (e.g., "Get a greeting for Mike") that uses the hello tool.
        model_id (str, optional): The model_id to use. Defaults to Mistral AI endpoint ('https://api.mistral.ai/v1/chat/completions').
        api_key (str, optional): The API key for the LLM provider. [Get a free API key from Mistral AI](https://console.mistral.ai/).

    Returns:
        str: The agent's response as a string, or an error message if something fails.
    """
    try:
        if not isinstance(prompt, str) or not prompt.strip():
            return "Invalid prompt. Please provide a non-empty string."

        # If api_key is None or set to a default value, use idToken from globals() if available
        default_keys = [None, '', 'your_mistral_api_key']
        if (api_key in default_keys) and ("idToken" in globals()):
            api_key = globals()["idToken"]
            model_id = "https://llm.boardflare.com"

        model = InferenceClientModel(
            model_id=model_id,
            api_key=api_key
        )

        @tool
        def hello(name: str) -> str:
            """
            Returns a greeting.

            Args:
                name: The name of the person that you want a greeting for.
            """
            return f"Hello {name}!"

        agent = CodeAgent(tools=[hello], model=model, add_base_tools=False)
        result = agent.run(prompt)
        if isinstance(result, str):
            return result
        return str(result)
    except Exception as e:
        return f"error: {e}"

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent / "test"))
from test_utils import get_graph_token

def inject_id_token():
    # Acquire token using shared utility
    token = get_graph_token()
    globals()["idToken"] = token

inject_id_token()

def test_basic_greeting():
    result = smolagents("Get a greeting for Mike")
    assert isinstance(result, str)
    assert any(word in result for word in ["Mike", "Hello", "greeting"])

def test_empty_prompt():
    result = smolagents("")
    assert isinstance(result, str)
    assert any(word in result.lower() for word in ["invalid", "error", "prompt"])

def test_non_string_prompt():
    result = smolagents(123)
    assert isinstance(result, str)
    assert any(word in result.lower() for word in ["invalid", "error", "prompt"])

ipytest.run()

In [ ]:
# Gradio Demo
import gradio as gr

def run_smolagents(prompt, model_id, api_key):
    return smolagents(prompt, model_id=model_id, api_key=api_key)

examples = [
    [
        "Get a greeting for Mike",
        "https://api.mistral.ai/v1/chat/completions",
        "your_mistral_api_key"
    ]
]

demo = gr.Interface(
    fn=run_smolagents,
    inputs=[
        gr.Textbox(label="Prompt", lines=2, value="Get a greeting for Mike"),
        gr.Textbox(label="Model ID", value="https://api.mistral.ai/v1/chat/completions"),
        gr.Textbox(label="API Key", value="your_mistral_api_key")
    ],
    outputs=gr.Textbox(label="Agent Response"),
    examples=examples,
    description="Demonstrates use of smolagents with a built-in HELLO tool.",
    flagging_mode="never",
)
demo.launch()